In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate

import pandas as pd

In [2]:
#импорт данных в датафреймы
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('../hw-2/ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('../hw-2/ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')


mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('../hw-2/ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python')

In [3]:
users.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [4]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
data=pd.merge(pd.merge(ratings,users),movies)
data.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   user_id     1000209 non-null  int64 
 1   movie_id    1000209 non-null  int64 
 2   rating      1000209 non-null  int64 
 3   timestamp   1000209 non-null  int64 
 4   gender      1000209 non-null  object
 5   age         1000209 non-null  int64 
 6   occupation  1000209 non-null  int64 
 7   zip         1000209 non-null  object
 8   title       1000209 non-null  object
 9   genres      1000209 non-null  object
dtypes: int64(6), object(4)
memory usage: 83.9+ MB


In [17]:
dataset = pd.DataFrame({
    'uid': data.user_id,
    'iid': data.title,
    'rating': data.rating
})
dataset.head()

,uid,iid,rating
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5


In [19]:
data.rating.min()

1

In [20]:
data.rating.max()

5

In [21]:
reader = Reader(rating_scale=(1.0, 5.0))
data_for_surprise = Dataset.load_from_df(dataset, reader)

In [22]:
algo = SVD()
cross_validate(algo, data_for_surprise, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8759  0.8714  0.8741  0.8750  0.8720  0.8737  0.0017  
Fit time          34.65   35.00   35.24   34.73   35.04   34.93   0.22    
Test time         1.70    1.65    1.50    1.67    1.65    1.63    0.07    


{'test_rmse': array([0.87585285, 0.87135515, 0.87412989, 0.87497466, 0.87201789]),
 'fit_time': (34.650084257125854,
  35.00426435470581,
  35.24384140968323,
  34.727133989334106,
  35.03990650177002),
 'test_time': (1.695897102355957,
  1.653381109237671,
  1.5023388862609863,
  1.6713769435882568,
  1.6473777294158936)}